In [1]:
import json
import pandas as pd
import secrets

In [2]:
def assign_rank(x):
    if pd.isna(x):
        return None
    return int(x // .1) + 1

def process_csv_ballots(csv_path, round_name):

    raw_voting_data = pd.read_csv(csv_path)

    dataframes = []
    for _, row in raw_voting_data.iterrows():
        if row['Status'] != 'SUBMITTED' and not row['Badgeholder']:
            continue
        if not isinstance(row['Payload'], str):
            continue

        payload = json.loads(row['Payload'])
        projects_payload = payload['project_allocations']
        
        addr = row['Address'].lower()
        budget = payload['budget']
        categories = payload['category_allocations']
        processed_voting_data = []
        for votes in projects_payload:
            project_id = list(votes.keys())[0].lower()
            project_vote = list(votes.values())[0]
            processed_voting_data.append({
                'voter_address': addr,
                'project_id': project_id,
                'vote': project_vote,
                'total_budget': budget,
                'category_allocations': categories
            })
        voter_dataframe = pd.DataFrame(processed_voting_data)
        dataframes.append(voter_dataframe)
        
    dataframe = pd.concat(dataframes, axis=0, ignore_index=True)
    #dataframe['project_score'] = pd.to_numeric(dataframe['vote']).rank(pct=True).apply(assign_rank)
    dataframe['round'] = round_name
    return dataframe

ballot_paths = [
    '../../../analysis/optimism/retrofunding5/data/_local/rf5_votes_final.csv',
    '../../../analysis/optimism/retrofunding6/data/_local/rf6_votes.csv'
]
voter_paths = [
    '../../../analysis/optimism/retrofunding5/data/_local/rf5_voters.csv',
    '../../../analysis/optimism/retrofunding6/data/_local/rf6_voters.csv',
]

In [3]:
df5 = pd.read_csv(voter_paths[0])
df5['voter_tags'] = df5.apply(lambda x: [x['Expertise group'], x['Voter Type']], axis=1)
df5['voter_address'] = df5['Wallet'].str.lower()
df5 = df5[['voter_address', 'voter_tags']]
df5 = process_csv_ballots(ballot_paths[0], 'RF5').merge(df5, on='voter_address')

voter_scramble = {v: f"0x{secrets.token_hex(20)}" for v in df5['voter_address'].unique()}
project_scramble = {p: f"0x{secrets.token_hex(32)}" for p in df5['project_id'].unique()}
df5['voter_address'] = df5['voter_address'].map(voter_scramble)
df5['project_id'] = df5['project_id'].map(project_scramble)

df5.tail(1)

,voter_address,project_id,vote,total_budget,category_allocations,round,voter_tags
2925,0xc6a7603665531d4322409fad66dee52e72c789a5,0xdb540243a45d1e2ad7f23f4ccc4c4114adea36d960ab...,1.27,6000000,"[{'OP_STACK_RESEARCH_AND_DEVELOPMENT': '50'}, ...",RF5,"[Expert, Guest Voter]"


In [4]:
df6 = pd.read_csv(voter_paths[1])
df6['voter_tags'] = df6.apply(lambda x: [x['Voter Type']], axis=1)
df6['voter_address'] = df6['Wallet'].str.lower()
df6 = df6[['voter_address', 'voter_tags']]
df6 = process_csv_ballots(ballot_paths[1], 'RF6').merge(df6, on='voter_address')

voter_scramble = {v: f"0x{secrets.token_hex(20)}" for v in df6['voter_address'].unique()}
project_scramble = {p: f"0x{secrets.token_hex(32)}" for p in df6['project_id'].unique()}
df6['voter_address'] = df6['voter_address'].map(voter_scramble)
df6['project_id'] = df6['project_id'].map(project_scramble)

df6.tail(1)

,voter_address,project_id,vote,total_budget,category_allocations,round,voter_tags
4169,0x3263235cc7514d387e27ec6145c2435709a12145,0x991129bf6985a6810b1fa966d1c6d0fdcd97b33bd44a...,None,3500000,"[{'GOVERNANCE_INFRA_AND_TOOLING': '33.34'}, {'...",RF6,[Citizen]


In [5]:
df = pd.concat([df5,df6], axis=0, ignore_index=True)
df.to_csv('rf_anon_votes.csv')